## 0. 빅쿼리 연동 및 확인 코드

- 테스트 데이터는 구글에서 제공하는 무료 데이터셋
- 아래와 같이 코드가 잘 나왔다면, 연동 완료

In [ ]:
# 라이브러리 설치
# pip install --upgrade google-cloud-bigquery

In [ ]:
from google.oauth2 import service_account
from google.cloud import bigquery

SERVICE_ACCOUNT_FILE = "./api_key.json"  # 키 json 파일
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)
project_id = "bigquery-test-408414"
client = bigquery.Client(credentials=credentials, project=project_id)

def import_bigquery_data(query):
    query_job = client.query(query)
    return query_job.to_dataframe()

data = import_bigquery_data('''
    SELECT *
    FROM `bigquery-public-data.google_trends.international_top_rising_terms`
    WHERE refresh_date = '2024-08-01'
    LIMIT 10
    ;
''')

data.head()

,term,rank,country_name,country_code,region_name,percent_gain,refresh_date,region_code,week,score
0,amor fati,20,Romania,RO,Alba County,200,2024-08-01,RO-AB,2019-08-11,<NA>
1,amor fati,20,Romania,RO,Alba County,200,2024-08-01,RO-AB,2019-09-08,<NA>
2,amor fati,20,Romania,RO,Alba County,200,2024-08-01,RO-AB,2020-01-26,<NA>
3,amor fati,20,Romania,RO,Alba County,200,2024-08-01,RO-AB,2020-02-16,<NA>
4,amor fati,20,Romania,RO,Alba County,200,2024-08-01,RO-AB,2020-03-15,<NA>


## 1. 이전에 활용한 쿼리결과를 파이썬으로 추출해보기

### 1-0. 필요한 데이터 미리 추출해주기
- 사용 테이블 : products, order_items, orders, users, events

In [ ]:
import pandas as pd

In [ ]:
import time
st = time.time()
products = import_bigquery_data("""
    SELECT * FROM `bigquery-public-data.thelook_ecommerce.products`
""")
et1 = time.time()
print('걸린시간1 : ',et1 - st)
order_items = import_bigquery_data("""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE created_at BETWEEN "2024-06-01" AND "2025-01-01"
""")
et2 = time.time()
print('걸린시간2 : ',et2 - st)
orders = import_bigquery_data("""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    WHERE created_at BETWEEN "2024-06-01" AND "2025-01-01"
""")
et3 = time.time()
print('걸린시간3 : ',et3 - st)
users = import_bigquery_data("""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.users`
""")
et4 = time.time()
print('걸린시간4 : ',et4 - st)
events = import_bigquery_data("""
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.events`
    WHERE created_at BETWEEN "2024-06-01" AND "2025-01-01"
""")
et5 = time.time()
print('걸린시간5 : ',et5 - st)

걸린시간1 :  13.193501234054565
걸린시간2 :  24.546009063720703
걸린시간3 :  29.858300924301147
걸린시간4 :  71.04166197776794
걸린시간5 :  197.17251300811768


In [ ]:
products = pd.read_csv('products.csv')
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
users = pd.read_csv('users.csv')
events = pd.read_csv('events.csv')

### 예시쿼리1

In [ ]:
# 예시쿼리 1
data1 = import_bigquery_data('''
    SELECT oi.product_id as product_id, p.name as product_name, p.category as product_category, count(*) as num_of_orders
    FROM `bigquery-public-data.thelook_ecommerce.products` as p
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi
    ON p.id = oi.product_id
    GROUP BY 1,2,3
    ORDER BY num_of_orders DESC
''')

data1.head()

,product_id,product_name,product_category,num_of_orders
0,24335,Schaefer The Original Duster,Outerwear & Coats,18
1,23293,Wrangler Men's Tallahassee Agility Short,Shorts,18
2,6682,Roxy Women's Spring Sets Linen Shorts Pink 468...,Shorts,17
3,24118,Levi's Men's Washed Cotton 2 Pocket Trucker Ja...,Outerwear & Coats,17
4,27815,parke & ronen Men's Taslan Inseam Swim Short,Swim,17


In [ ]:
# merge
tmp_merge_data = pd.merge(
    products[['id','name','category']],
    order_items[['product_id']],
    how='inner',
    left_on='id',
    right_on='product_id'
)
# group by
tmp_group_data = tmp_merge_data.groupby(['product_id','name','category']).count()
# sort
result1 = tmp_group_data.sort_values(by='id', ascending=False).reset_index()

In [ ]:
result1.head()

,product_id,name,category,id
0,16746,Lucky Brand Mens Men's Vintage Slub Double Poc...,Tops & Tees,7
1,26646,Mens Plaid Print Fleece Pajama Pants,Sleep & Lounge,7
2,8941,Hue Womens Textured Diamond Tight,Socks & Hosiery,6
3,27312,Robinson Apparel Unisex Button-Fly Collegiate ...,Sleep & Lounge,6
4,14459,Ingrid & Isabel Women's Everywear Pant,Maternity,6


### 예시쿼리2


In [ ]:
data2 = import_bigquery_data('''
    SELECT u.id as user_id, u.first_name, u.last_name, avg(oi.sale_price) as avg_sale_price
    FROM `bigquery-public-data.thelook_ecommerce.users` as u
    JOIN `bigquery-public-data.thelook_ecommerce.order_items` as oi
    ON u.id = oi.user_id
    GROUP BY 1,2,3
    ORDER BY avg_sale_price DESC
    LIMIT 10
''')
data2.head()

,user_id,first_name,last_name,avg_sale_price
0,98429,Timothy,Rojas,999.0
1,77859,Henry,Castro,999.0
2,93221,Roger,Lewis,999.0
3,5918,Joe,Meyer,999.0
4,76349,Michael,Shannon,950.0


In [ ]:
# merge
tmp_merge_data = pd.merge(
    users[['id','first_name','last_name']],
    order_items[['user_id','sale_price']],
    how='inner',
    left_on = 'id',
    right_on = 'user_id'
)

# GROUP
tmp_group_data = tmp_merge_data.groupby(['id','first_name','last_name'])[['sale_price']].mean()

# sort
result2 = tmp_group_data.sort_values(by='sale_price', ascending=False).reset_index()

In [ ]:
result2.head()

,id,first_name,last_name,sale_price
0,77859,Henry,Castro,999.0
1,493,Luke,Moore,903.0
2,55782,Jose,Hill,903.0
3,87918,Ryan,Thornton,903.0
4,58788,George,Taylor,903.0


### Q1. 2023년 가입 유저수

In [ ]:
data3 = import_bigquery_data('''
    SELECT COUNT(id)
    FROM `bigquery-public-data.thelook_ecommerce.users`
    WHERE created_at BETWEEN "2024-06-01" AND "2025-01-01"
    ;
''')

data3.head()

,f0_
0,5903


In [ ]:
len(users)

100000

### Q2. 브라우저별 세션 수

In [ ]:
data4 = import_bigquery_data('''
    SELECT
     browser,
     COUNT(session_id) AS total_session
    FROM `bigquery-public-data.thelook_ecommerce.events`
    WHERE sequence_number = 1 AND created_at BETWEEN "2024-06-01" AND "2025-01-01"
    GROUP BY browser
    ORDER BY total_session DESC
    ;
''')

data4.head()

,browser,total_session
0,Chrome,24804
1,Firefox,10044
2,Safari,9991
3,Other,2511
4,IE,2402


In [ ]:
events[events['sequence_number']==1].groupby('browser')[['session_id']].count().sort_values(by='session_id', ascending=False)

,session_id
browser,
Chrome,24804
Firefox,10044
Safari,9991
Other,2511
IE,2402


### Q3. 2023년 월별 매출액(sale_price*num_of_item), 주문수, 주문 유저수 계산

In [ ]:
data5 = import_bigquery_data('''
    SELECT
     EXTRACT(MONTH FROM A.created_at) AS month,
     SUM(A.sale_price*B.num_of_item) AS revenue,
     COUNT(DISTINCT A.order_id) AS order_count,
     COUNT(DISTINCT A.user_id) AS customers_purchased
    FROM `bigquery-public-data.thelook_ecommerce.order_items` AS A
    LEFT OUTER JOIN `bigquery-public-data.thelook_ecommerce.orders` AS B ON A.order_id = B.order_id
    WHERE A.status NOT IN ('Cancelled','Returned') AND A.created_at BETWEEN "2024-06-01" AND "2025-01-01"
    GROUP BY EXTRACT(MONTH FROM A.created_at)
    ORDER BY month
    ;
''')

data5.head()

,month,revenue,order_count,customers_purchased
0,6,811791.240907,4920,4700
1,7,981245.240761,6131,5661
2,8,933758.030872,5666,4600


In [ ]:
# merge
tmp_merge_data = pd.merge(
    order_items[['order_id','created_at','user_id','status','sale_price']],
    orders[['order_id','num_of_item']],
    how='left',
    on='order_id'
)
# 조건
tmp_merge_data = tmp_merge_data[~tmp_merge_data['status'].isin(['Cancelled','Returned'])]

# month 값 생성
tmp_merge_data['month'] = [i[5:7] for i in tmp_merge_data['created_at']]

# group by
result5 = tmp_merge_data.groupby('month').agg(
    revenue=('sale_price', lambda x: (x * tmp_merge_data.loc[x.index, 'num_of_item']).sum()),
    order_count=('order_id', 'nunique'),
    customers_purchased=('user_id', 'nunique')
).reset_index()

result5.head()

,month,revenue,order_count,customers_purchased
0,06,777098.870859,4920,4700
1,07,981245.240761,6131,5661
2,08,933758.030872,5666,4600


### Q4. order_items에서 status가 Cancelled, Returned된 상품들의 남녀로 나누어서 매출액(sale_price*num_of_item), 수량 계산하기

In [ ]:
data6 = import_bigquery_data('''
    SELECT
     B.gender,
     SUM(A.sale_price*B.num_of_item) AS revenue,
     SUM(B.num_of_item) quantity
    FROM `bigquery-public-data.thelook_ecommerce.order_items` A
    LEFT JOIN `bigquery-public-data.thelook_ecommerce.orders` B ON A.order_id = B.order_id
    WHERE A.status NOT IN ('Cancelled','Returned')
    GROUP BY B.gender
    ORDER BY revenue
    ;
''')

data6.head()

,gender,revenue,quantity
0,F,7.272677e+06,129167
1,M,8.127655e+06,129288


In [ ]:
# merge
tmp_merge_data = pd.merge(
    order_items[['order_id','created_at','user_id','status','sale_price']],
    orders[['order_id','num_of_item','gender']],
    how='left',
    on='order_id'
)

# 조건
tmp_merge_data = tmp_merge_data[~tmp_merge_data['status'].isin(['Cancelled','Returned'])]

# 칼럼 추가
tmp_merge_data['revenue'] = tmp_merge_data['sale_price']*tmp_merge_data['num_of_item']
result6 = tmp_merge_data.groupby('gender')[['revenue','num_of_item']].sum()

result6.head()

,revenue,num_of_item
gender,,
F,1.258278e+06,22381.0
M,1.433824e+06,23058.0


### Q5. TRAFFIC SOURCE 별 전체 sessions, 구매 sessions, 구매 sessions/전체 sessions(세션별 구매율)

In [ ]:
data7 = import_bigquery_data('''
    WITH a AS (
     SELECT
       traffic_source,
       COUNT(session_id) AS total_session
     FROM `bigquery-public-data.thelook_ecommerce.events`
     WHERE sequence_number = 1 AND created_at BETWEEN "2024-06-01" AND "2025-01-01"
     GROUP BY traffic_source
    ),b AS (
     SELECT
       traffic_source,
       COUNT(session_id) As purchase_session
     FROM `bigquery-public-data.thelook_ecommerce.events`
     WHERE event_type= 'purchase' AND session_id IN
       (SELECT session_id
       FROM `bigquery-public-data.thelook_ecommerce.events`
       GROUP BY session_id
       HAVING MIN(created_at) BETWEEN "2024-06-01" AND "2025-01-01")
     GROUP BY traffic_source
    )

    SELECT
     a.traffic_source,
     a.total_session,
     b.purchase_session,
     ROUND(SAFE_DIVIDE(b.purchase_session, a.total_session), 4) AS conversion_rate
    FROM a JOIN b ON a.traffic_source = b.traffic_source
    ORDER BY purchase_session DESC
    ;
''')

data7.head()

,traffic_source,total_session,purchase_session,conversion_rate
0,Email,22235,14246,0.6407
1,Adwords,15068,9703,0.6439
2,YouTube,5011,3222,0.6430
3,Facebook,4913,3114,0.6338
4,Organic,2525,1612,0.6384


### 참고 : chatgpt 활용한 파이썬 코드

In [ ]:
# 데이터 전처리
events['created_at'] = pd.to_datetime(events['created_at'], errors='coerce')

# A 서브쿼리: 총 세션 수
a = (events[(events['sequence_number'] == 1) & (events['created_at'] >= '2024-06-01') & (events['created_at'] < '2025-01-01')]
     .groupby('traffic_source')
     .agg(total_session=('session_id', 'count'))
     .reset_index()
)

# B 서브쿼리: 구매 세션 수
purchase_sessions = (events[(events['event_type'] == 'purchase') &
                            (events['created_at'] >= '2024-06-01') & (events['created_at'] < '2025-01-01')]
                     .drop_duplicates(subset='session_id')
                     .groupby('session_id')
                     .size()
                     .reset_index(name='purchase_count')
)

# 구매 세션과 관련된 traffic_source 가져오기
purchase_sessions_with_source = (events[events['session_id'].isin(purchase_sessions['session_id'])]
                                 .groupby('traffic_source')
                                 .agg(purchase_session=('session_id', 'nunique'))
                                 .reset_index()
)

# A와 B 서브쿼리 결합
result = pd.merge(a, purchase_sessions_with_source, on='traffic_source', how='left')

# conversion_rate 계산
result['conversion_rate'] = result['purchase_session'] / result['total_session']
result['conversion_rate'] = result['conversion_rate'].round(4)

# 결괏값 정렬
result = result.sort_values(by='purchase_session', ascending=False)

In [ ]:
result.head()

,traffic_source,total_session,purchase_session,conversion_rate
1,Email,20168,12317,0.6107
0,Adwords,13714,8441,0.6155
4,YouTube,4561,2791,0.6119
2,Facebook,4446,2680,0.6028
3,Organic,2270,1363,0.6004
